In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.utils import class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score as f1_score_sk
import scikitplot as skplt

from lime import lime_image

from modules.main_model import create_model
from utils.image_manipulation import resize_and_pad, swap_labels
from utils.loss_and_scoring import FocalLoss, f1_score

In [ ]:
device = tf.config.list_physical_devices('GPU')
print(f"Using device: {device}")

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
])

preprocess_input = tf.keras.applications.efficientnet_v2.preprocess_input

IMG_SIZE = (300, 300)
BATCH_SIZE = 64

In [4]:
def make_metrics_plots(y_test, y_pred, y_pred_proba):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    skplt.metrics.plot_confusion_matrix(y_test, y_pred, ax=axes[0, 0])
    skplt.metrics.plot_roc(y_test, y_pred_proba, ax=axes[0, 1])
    skplt.metrics.plot_precision_recall(y_test, y_pred_proba, ax=axes[1, 0])
    skplt.metrics.plot_cumulative_gain(y_test, y_pred_proba, ax=axes[1, 1])

    plt.show()

In [ ]:
train_dir = 'data/train'
test_dir = 'data/test'

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, seed=50)
test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
train_dataset = train_dataset.map(resize_and_pad).map(swap_labels)
test_dataset = test_dataset.map(resize_and_pad).map(swap_labels)

In [ ]:
fraud_images = []
non_fraud_images = []

for images, labels in train_dataset:
    for i in range(len(labels)):
        if labels[i] == 1:
            fraud_images.append(images[i].numpy().astype("uint8"))
        elif labels[i] == 0:
            non_fraud_images.append(images[i].numpy().astype("uint8"))

        if len(fraud_images) >= 13 and len(non_fraud_images) >= 12:
            break
        
    if len(fraud_images) >= 13 and len(non_fraud_images) >= 12:
        break

plt.figure(figsize=(10, 10))

for i in range(13):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(fraud_images[i])
    plt.title("Fraud")
    plt.axis("off")

for i in range(12):
    ax = plt.subplot(5, 5, i + 14)
    plt.imshow(non_fraud_images[i])
    plt.title("Non-Fraud")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
train_labels = np.concatenate([y.numpy() for _, y in train_dataset])
pred_labels = np.concatenate([y.numpy() for _, y in test_dataset])
all_labels = np.concatenate((train_labels, pred_labels))

class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(all_labels), y=all_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weight_dict)

## Creating Model

In [9]:
model = create_model(num_classes=2)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=FocalLoss(),
              metrics=['accuracy', f1_score])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='f1_score', patience=5, restore_best_weights=True, mode='max')

model.summary()

In [10]:
history = model.fit(train_dataset,
                    epochs=10,
                    callbacks=[early_stopping],
                    class_weight=class_weight_dict)

In [11]:
# model.save('models/cnn_fraud_model_effv2b3.keras')

In [12]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].plot(history.history['accuracy'], label='Accuracy')
ax[0].plot(history.history['f1_score'], label='F1 Score')
ax[0].set_title('Accuracy and F1 Score')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Score')
ax[0].legend()

ax[1].plot(history.history['loss'], label='Loss', color='red')
ax[1].set_title('Loss')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].legend()

plt.show()

In [13]:
images, labels = [], []
for batch_images, batch_labels in train_dataset:
    images.append(batch_images.numpy())
    labels.append(batch_labels.numpy())

images = np.concatenate(images, axis=0)
labels = np.concatenate(labels, axis=0)

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

acc_scores = []
f1_scores_l = []

for train_index, val_index in kf.split(images):
    X_train, X_val = images[train_index], images[val_index]
    y_train, y_val = labels[train_index], labels[val_index]

    model = create_model(num_classes=2)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=FocalLoss(),
                  metrics=['accuracy', f1_score])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='f1_score', patience=5, restore_best_weights=True, mode='max')

    history = model.fit(train_dataset,
                    epochs=10,
                    callbacks=[early_stopping],
                    class_weight=class_weight_dict)

    score = model.evaluate(X_val, y_val, verbose=0)
    acc_scores.append(score[1])
    f1_scores_l.append(score[2])

In [14]:
average_accuracy = np.mean(acc_scores)
f1_scores = np.mean(f1_scores_l)
print(f'Average Accuracy across {k} folds: {average_accuracy:.4f}, {f1_scores:.4f}')

In [16]:
keras_tuners = pd.read_csv('keras_tuner_results.csv')

In [ ]:
keras_tuners[['num_dense_layers', 'learning_rate', 'train_accuracy', 'val_accuracy', 'train_f1_score', 
              'val_f1_score', 'train_loss', 'val_loss']].sort_values('val_f1_score', ascending=False).head(10)

In [18]:
# model = keras.models.load_model('models/cnn_fraud_model_effv2b3.keras', custom_objects={'FocalLoss': FocalLoss, 'f1_score': f1_score})

In [ ]:
model.summary()

## Testing Model

In [ ]:
test_predictions = model.predict(test_dataset)

In [21]:
raw_labels = []

for images, labels in test_dataset:
    raw_labels.append(labels.numpy())

true_labels = np.concatenate(raw_labels)

In [ ]:
sns.histplot(x=test_predictions[:, 1], hue=true_labels, stat="density", common_norm=False)
plt.title('Prediction Probabilities vs True Labels')
plt.show()

In [23]:
df_threshold = pd.DataFrame()

for i, th in enumerate(np.arange(0.4, 0.61, 0.01)):
    test_labels_l =  (test_predictions > th).astype(int)[:, 1]
    df_threshold.loc[i, 'threshold'] = th
    df_threshold.loc[i, 'accuarcy'] = accuracy_score(true_labels, test_labels_l)
    df_threshold.loc[i, 'precision'] = precision_score(true_labels, test_labels_l)
    df_threshold.loc[i, 'recall'] = recall_score(true_labels, test_labels_l)
    df_threshold.loc[i, 'f1_score'] = f1_score_sk(true_labels, test_labels_l)

In [ ]:
round(df_threshold.sort_values(['f1_score'], ascending=False).head(), 3)

In [25]:
pred_labels = (test_predictions > 0.53).astype(int)[:, 1]

In [ ]:
print(classification_report(true_labels, pred_labels))

In [ ]:
make_metrics_plots(true_labels, pred_labels, test_predictions)

In [ ]:
fraud_images = []
non_fraud_images = []
fraud_labels = []
non_fraud_labels = []

for images, true_labels_img in test_dataset:
    true_labels_img = true_labels_img.numpy()

    for i in range(len(images)):
        if true_labels_img[i] == 1 and pred_labels[i] == 1 and len(fraud_images) < 13:
            fraud_images.append(images[i].numpy().astype("uint8"))
            fraud_labels.append(pred_labels[i])

        elif true_labels_img[i] == 0 and pred_labels[i] == 0 and len(non_fraud_images) < 12:
            non_fraud_images.append(images[i].numpy().astype("uint8"))
            non_fraud_labels.append(pred_labels[i])

        if len(fraud_images) >= 13 and len(non_fraud_images) >= 12:
            break

    if len(fraud_images) >= 12 and len(non_fraud_images) >= 12:
        break

plt.figure(figsize=(10, 10))

for i in range(13):
    ax = plt.subplot(5, 5, i + 1)
    plt.imshow(fraud_images[i])
    plt.title(f"Predicted: {fraud_labels[i]}, True: 1")
    plt.axis("off")

for i in range(12):
    ax = plt.subplot(5, 5, i + 14)
    plt.imshow(non_fraud_images[i])
    plt.title(f"Predicted: {non_fraud_labels[i]}, True: 0")
    plt.axis("off")

plt.tight_layout()
plt.show()

## Lime For Explainability

In [29]:
fraud_images = []
non_fraud_images = []
fraud_labels = []
non_fraud_labels = []

for images, true_labels_img in test_dataset:
    true_labels_img = true_labels_img.numpy()

    for i in range(len(images)):
        if true_labels_img[i] == 1 and pred_labels[i] == 1 and len(fraud_images) < 3:
            fraud_images.append(images[i].numpy().astype("uint8"))
            fraud_labels.append(pred_labels[i])

        elif true_labels_img[i] == 0 and pred_labels[i] == 0 and len(non_fraud_images) < 3:
            non_fraud_images.append(images[i].numpy().astype("uint8"))
            non_fraud_labels.append(pred_labels[i])

        if len(fraud_images) >= 3 and len(non_fraud_images) >= 3:
            break

    if len(fraud_images) >= 3 and len(non_fraud_images) >= 3:
        break

explainer = lime_image.LimeImageExplainer()

for i in range(3):
    explanation_fraud = explainer.explain_instance(fraud_images[i], lambda x: model.predict(x, verbose=0), 
                                                   top_labels=5, hide_color=0, num_samples=1000, random_seed=50)

    top_label_fraud = explanation_fraud.top_labels[0]
    temp_fraud, mask_fraud = explanation_fraud.get_image_and_mask(top_label_fraud, positive_only=True, num_features=10, hide_rest=False)

    temp_fraud = (temp_fraud - temp_fraud.min()) / (temp_fraud.max() - temp_fraud.min())
    temp_fraud = (temp_fraud * 255).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(fraud_images[i])
    plt.title(f"Fraud - Predicted: {fraud_labels[i]}, True: 1")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(temp_fraud)
    plt.imshow(mask_fraud, alpha=0.5, cmap='jet')
    plt.title(f"LIME: {top_label_fraud}")
    plt.axis("off")

    plt.show()


for i in range(3):
    explanation_non_fraud = explainer.explain_instance(non_fraud_images[i], lambda x: model.predict(x, verbose=0),
                                                       top_labels=5, hide_color=0, num_samples=1000, random_seed=50)

    top_label_non_fraud = explanation_non_fraud.top_labels[0]
    temp_non_fraud, mask_non_fraud = explanation_non_fraud.get_image_and_mask(top_label_non_fraud, positive_only=True, num_features=10, hide_rest=False)

    temp_non_fraud = (temp_non_fraud - temp_non_fraud.min()) / (temp_non_fraud.max() - temp_non_fraud.min())
    temp_non_fraud = (temp_non_fraud * 255).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(non_fraud_images[i])
    plt.title(f"Non-Fraud - Predicted: {non_fraud_labels[i]}, True: 0")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(temp_non_fraud)
    plt.imshow(mask_non_fraud, alpha=0.5, cmap='jet')
    plt.title(f"LIME: {top_label_non_fraud}")
    plt.axis("off")

    plt.show()